In [12]:
!pip install tensorflow
import tensorflow as tf



In [13]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

#loading flower dataset
dataset, info = tfds.load('tf_flowers', as_supervised=True, with_info=True)
dataset_size = info.splits['train'].num_examples
class_names = info.features['label'].names
n_classes = info.features['label'].num_classes




In [14]:
test_set_raw, valid_set_raw, train_set_raw = tfds.load(
    'tf_flowers', split = ['train[:10%]', 'train[10%:25%]', 'train[25%:]'],
    as_supervised=True
)


In [15]:
batch_size= 32
preprocess = tf.keras.Sequential([
    tf.keras.layers.Resizing(height=224, width=224, crop_to_aspect_ratio=True),
    tf.keras.layers.Lambda(tf.keras.applications.xception.preprocess_input)
])
train_set= train_set_raw.map(lambda X,y: (preprocess(X),y))
train_set = train_set.shuffle(1000, seed=42).batch(batch_size)
valid_set = valid_set_raw.map(lambda X, y: (preprocess(X), y)).batch(batch_size)
test_set = test_set_raw.map(lambda X,y: (preprocess(X),y)).batch(batch_size)


In [16]:
base_model = tf.keras.applications.xception.Xception(weights = 'imagenet',
                                                include_top = False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(n_classes,activation='softmax')(avg)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [17]:
for layer in base_model.layers:
  layer.trainable=False

In [18]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics=['accuracy'])
history = model.fit(train_set, validation_data=valid_set, epochs=3)

Epoch 1/3
86/86 [==============================] - 10s 55ms/step - loss: 0.7506 - accuracy: 0.8143 - val_loss: 0.6830 - val_accuracy: 0.8439
Epoch 2/3
86/86 [==============================] - 4s 42ms/step - loss: 0.3272 - accuracy: 0.9066 - val_loss: 0.5392 - val_accuracy: 0.8657
Epoch 3/3
86/86 [==============================] - 4s 42ms/step - loss: 0.2148 - accuracy: 0.9324 - val_loss: 0.5459 - val_accuracy: 0.8693


In [21]:
for layer in base_model.layers[56:]:
  layer.trainable = True

In [22]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer,
              metrics = ['accuracy'])
history = model.fit(train_set, validation_data = valid_set, epochs=10)


Epoch 1/10
86/86 [==============================] - 15s 56ms/step - loss: 0.2686 - accuracy: 0.9081 - val_loss: 0.6032 - val_accuracy: 0.8530
Epoch 2/10
86/86 [==============================] - 4s 45ms/step - loss: 0.0605 - accuracy: 0.9847 - val_loss: 0.3279 - val_accuracy: 0.9074
Epoch 3/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0144 - accuracy: 0.9953 - val_loss: 0.3569 - val_accuracy: 0.9129
Epoch 4/10
86/86 [==============================] - 4s 46ms/step - loss: 0.0097 - accuracy: 0.9971 - val_loss: 0.3352 - val_accuracy: 0.9165
Epoch 5/10
86/86 [==============================] - 4s 45ms/step - loss: 0.0106 - accuracy: 0.9975 - val_loss: 0.3651 - val_accuracy: 0.9201
Epoch 6/10
86/86 [==============================] - 4s 45ms/step - loss: 0.0069 - accuracy: 0.9982 - val_loss: 0.3236 - val_accuracy: 0.9147
Epoch 7/10
86/86 [==============================] - 4s 45ms/step - loss: 0.0035 - accuracy: 0.9989 - val_loss: 0.3416 - val_accuracy: 0.9129
Epoch 8/10
8